In [87]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from docplex.cp.model import  CpoModel
import docplex.cp.utils_visu as visu

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

import matplotlib.pyplot as plt
%matplotlib inline
#Change the plot size
from pylab import rcParams
rcParams['figure.figsize'] = 15, 3

import sys
sys.path.append("..")
from utils import extract_solution

Taillard specification

On the first line are two numbers, the first is the number of jobs and the second the number of machines. Following there are two matrice the first with a line for each job containing the processor times for each operation the second with the order for visiting the machines. The numbering of the machines starts at 1.

For example the same instance as above would be presented as:

1	3	

6	7	5

2	3	1



In [276]:
Text =[]
with open("data/abz5.txt") as f:
    for line in f:
        Text.append(line)

firstline = Text[0].split('\t')
num_jobs = int(firstline[0])
num_machines = int ( firstline[1].replace('\n',"") )

durations =[]
for i in range(1,num_jobs+1):
    a= Text[i].split('\t')
    durations.append( [int(j) for j in a])
durations = pd.DataFrame(durations)
display(durations)


machines =[]
for i in range(num_jobs+1,2*num_jobs+1):
    a= Text[i].split('\t')
    machines.append( [int(j) for j in a])
machines = pd.DataFrame(machines)
display(machines)

,0,1,2,3,4,5,6,7,8,9
0,88,68,94,99,67,89,77,99,86,92
1,72,50,69,75,94,66,92,82,94,63
2,83,61,83,65,64,85,78,85,55,77
3,94,68,61,99,54,75,66,76,63,67
4,69,88,82,95,99,67,95,68,67,86
5,99,81,64,66,80,80,69,62,79,88
6,50,86,97,96,95,97,66,99,52,71
7,98,73,82,51,71,94,85,62,95,79
8,94,71,81,85,66,90,76,58,93,97
9,50,59,82,67,56,96,58,81,59,96


,0,1,2,3,4,5,6,7,8,9
0,5,9,7,6,2,3,10,8,1,4
1,6,4,7,5,3,9,1,2,8,10
2,10,9,1,2,7,6,8,5,3,4
3,8,3,2,5,4,7,6,1,10,9
4,4,5,10,9,1,3,7,6,8,2
5,2,5,6,7,9,3,8,10,4,1
6,8,2,5,4,1,9,3,6,7,10
7,5,7,4,3,2,6,8,1,9,10
8,1,7,4,8,2,3,5,6,9,10
9,4,1,2,9,8,10,7,5,6,3


In [277]:
durations = \
durations.melt(ignore_index=False).reset_index(drop = False)
data = durations.rename(columns = {"index":"job",'variable':"operation","value":"duration"}).sort_values(['job','operation'])
data.head()

,job,operation,duration
0,0,0,88
10,0,1,68
20,0,2,94
30,0,3,99
40,0,4,67


In [278]:
machines = machines.melt(ignore_index=False).reset_index(drop = False).rename(columns = {"index":"job",'variable':"operation","value":"machine"}).reset_index(drop = True).sort_values(['job','operation'])
machines.head()

,job,operation,machine
0,0,0,5
10,0,1,9
20,0,2,7
30,0,3,6
40,0,4,2


In [279]:
data = pd.merge(data, machines, on= ['job','operation']).sort_values(['job','operation'])
data.head()

,job,operation,duration,machine
0,0,0,88,5
1,0,1,68,9
2,0,2,94,7
3,0,3,99,6
4,0,4,67,2


In [280]:
horizon = data['duration'].sum()

mdl = CpoModel(name="Taillard")

DV=[]
for row in data.itertuples():

    dv = mdl.interval_var((0, horizon) , (0, horizon) , size=row.duration, name="Job{}:Oper{}:Mach{}".format(row.job, row.operation, row.machine))
    DV.append(dv) 
data['DV'] = DV
data.head()

,job,operation,duration,machine,DV
0,0,0,88,5,"""Job0:Oper0:Mach5"" = intervalVar(start=0..7773, end=0..7773, size=88)"
1,0,1,68,9,"""Job0:Oper1:Mach9"" = intervalVar(start=0..7773, end=0..7773, size=68)"
2,0,2,94,7,"""Job0:Oper2:Mach7"" = intervalVar(start=0..7773, end=0..7773, size=94)"
3,0,3,99,6,"""Job0:Oper3:Mach6"" = intervalVar(start=0..7773, end=0..7773, size=99)"
4,0,4,67,2,"""Job0:Oper4:Mach2"" = intervalVar(start=0..7773, end=0..7773, size=67)"


In [281]:
# machines are allocated to one job at a time

jobs_same_machine = data[['machine','DV']].groupby('machine').agg(list).reset_index(drop=False)

display(jobs_same_machine.head(1))

for row in jobs_same_machine.itertuples():
    mdl.add ( mdl.no_overlap(  row.DV ))

,machine,DV
0,1,"[""Job0:Oper8:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=86), ""Job1:Oper6:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=92), ""Job2:Oper2:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=83), ""Job3:Oper7:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=76), ""Job4:Oper4:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=99), ""Job5:Oper9:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=88), ""Job6:Oper4:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=95), ""Job7:Oper7:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=62), ""Job8:Oper0:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=94), ""Job9:Oper1:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=59)]"


In [282]:
# precedence
jobs = data['job'].unique()

for job in jobs:

    u = data[data['job']==job].reset_index(drop=True)
    u['order']= range(1, len(u)+1)
    m = len(u)

    precedence_list = [[i,i+1] for i in range(1,m+1)][:-1]
    for i in precedence_list:
        first_task = u[u['order']==i[0]]['DV'].values[0]
        second_task = u[u['order']==i[1]]['DV'].values[0]
        #print( first_task,second_task )
        mdl.add( mdl.end_before_start(first_task  , second_task ))

"Job0:Oper0:Mach5" = intervalVar(start=0..7773, end=0..7773, size=88) "Job0:Oper1:Mach9" = intervalVar(start=0..7773, end=0..7773, size=68)
"Job0:Oper1:Mach9" = intervalVar(start=0..7773, end=0..7773, size=68) "Job0:Oper2:Mach7" = intervalVar(start=0..7773, end=0..7773, size=94)
"Job0:Oper2:Mach7" = intervalVar(start=0..7773, end=0..7773, size=94) "Job0:Oper3:Mach6" = intervalVar(start=0..7773, end=0..7773, size=99)
"Job0:Oper3:Mach6" = intervalVar(start=0..7773, end=0..7773, size=99) "Job0:Oper4:Mach2" = intervalVar(start=0..7773, end=0..7773, size=67)
"Job0:Oper4:Mach2" = intervalVar(start=0..7773, end=0..7773, size=67) "Job0:Oper5:Mach3" = intervalVar(start=0..7773, end=0..7773, size=89)
"Job0:Oper5:Mach3" = intervalVar(start=0..7773, end=0..7773, size=89) "Job0:Oper6:Mach10" = intervalVar(start=0..7773, end=0..7773, size=77)
"Job0:Oper6:Mach10" = intervalVar(start=0..7773, end=0..7773, size=77) "Job0:Oper7:Mach8" = intervalVar(start=0..7773, end=0..7773, size=99)
"Job0:Oper7:Mach8"

In [283]:
data.loc[:,'endtime']=data['DV'].apply(lambda x:mdl.end_of(x))

obj = mdl.minimize ( mdl.max ( data['endtime']) )

mdl.add(obj)

In [284]:
msol = mdl.solve(TimeLimit=50, trace_log=False)

msol.get_objective_values()

(1234,)

In [285]:
data['DV_sol'] = data['DV'].apply(lambda x:msol.get_var_solution(x))
data['DV_sol_start']=data['DV_sol'].apply(lambda x:x.get_start())
data['DV_sol_end']=data['DV_sol'].apply(lambda x:x.get_end())

data.sort_values(['DV_sol_start','DV_sol_end']).tail()

,job,operation,duration,machine,DV,endtime,DV_sol,DV_sol_start,DV_sol_end
69,6,9,71,10,"""Job6:Oper9:Mach10"" = intervalVar(start=0..7773, end=0..7773, size=71)","endOf(""Job6:Oper9:Mach10"")","Job6:Oper9:Mach10: (start=1100, end=1171, size=71, length=71)",1100,1171
99,9,9,96,3,"""Job9:Oper9:Mach3"" = intervalVar(start=0..7773, end=0..7773, size=96)","endOf(""Job9:Oper9:Mach3"")","Job9:Oper9:Mach3: (start=1124, end=1220, size=96, length=96)",1124,1220
59,5,9,88,1,"""Job5:Oper9:Mach1"" = intervalVar(start=0..7773, end=0..7773, size=88)","endOf(""Job5:Oper9:Mach1"")","Job5:Oper9:Mach1: (start=1125, end=1213, size=88, length=88)",1125,1213
9,0,9,92,4,"""Job0:Oper9:Mach4"" = intervalVar(start=0..7773, end=0..7773, size=92)","endOf(""Job0:Oper9:Mach4"")","Job0:Oper9:Mach4: (start=1125, end=1217, size=92, length=92)",1125,1217
19,1,9,63,10,"""Job1:Oper9:Mach10"" = intervalVar(start=0..7773, end=0..7773, size=63)","endOf(""Job1:Oper9:Mach10"")","Job1:Oper9:Mach10: (start=1171, end=1234, size=63, length=63)",1171,1234


In [286]:

colors ={'0':'yellow',   '1':'lightblue', '2':  'lightgreen','3' :'cyan' ,'4':'linen','5':'green',"6":'ivory'}
rcParams['figure.figsize'] = 20, 10

for i in data['DV'].values:
    wt = msol.get_var_solution(i)
    job = wt.get_name().split(':')[0].replace("Job","J")
    operation = wt.get_name().split(':')[1].replace("Oper","O")
    machine = wt.get_name().split(':')[2].replace("Mach","M")

    visu.interval(wt, colors[machine.replace("M","")], ':'.join([job, operation, machine])) 

visu.show()


KeyError: '9'

In [ ]:
# rows are Job Ids and columns are operations ids

processing_time = [[54,34,61,2],[9,15,89,70],[38,19,28,87],[95,34,7,29]]
processing_time = pd.DataFrame(processing_time, columns = range(1,5), index = range(1,5))

display(processing_time)

# rows are Job Ids and columns are operations ids
machines = [[3,1,4,2],[4,1,2,3],[1,2,3,4],[1,3,2,4]]
machines = pd.DataFrame(machines, columns = range(1,5), index = range(1,5))

display(machines)